In [33]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

In [19]:
# Import data
data = pd.read_csv('data_stocks.csv')
# Drop date variable
data = data.drop(['DATE'], 1)
# Dimensions of dataset
n = data.shape[0]
p = data.shape[1]
# Make data a numpy array
df = data
data = data.values

In [20]:
# Training and test data
train_start = 0
train_end = int(np.floor(0.8*n))
test_start = train_end
test_end = n
data_train = data[np.arange(train_start, train_end), :]
data_test = data[np.arange(test_start, test_end), :]


In [21]:
# Scale data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_train = scaler.fit_transform(data_train)
data_test = scaler.transform(data_test)
# Build X and y
X_train = data_train[:, 1:]
y_train = data_train[:, 0]
X_test = data_test[:, 1:]
y_test = data_test[:, 0]

In [43]:
model = Sequential()
model.add(Dense(1024, input_dim=500, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='relu'))

In [44]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [45]:
model.fit(X_train, y_train, epochs=10, batch_size=256)

Epoch 1/10
33012/33012 [==============================] - 4s 109us/step - loss: 0.0291 - mse: 0.0291
Epoch 2/10
33012/33012 [==============================] - 3s 99us/step - loss: 5.0241e-04 - mse: 5.0241e-04
Epoch 3/10
33012/33012 [==============================] - 3s 99us/step - loss: 2.3234e-04 - mse: 2.3234e-04
Epoch 4/10
33012/33012 [==============================] - 3s 98us/step - loss: 2.5467e-04 - mse: 2.5467e-04
Epoch 5/10
33012/33012 [==============================] - 3s 98us/step - loss: 2.4209e-04 - mse: 2.4209e-04
Epoch 6/10
33012/33012 [==============================] - 3s 99us/step - loss: 1.4439e-04 - mse: 1.4439e-04
Epoch 7/10
33012/33012 [==============================] - 3s 98us/step - loss: 2.2938e-04 - mse: 2.2938e-04
Epoch 8/10
33012/33012 [==============================] - 3s 98us/step - loss: 8.9373e-05 - mse: 8.9373e-05
Epoch 9/10
33012/33012 [==============================] - 3s 101us/step - loss: 8.8392e-05 - mse: 8.8392e-05 1s - los
Epoch 10/10
33012/33012 [

In [47]:
_, accuracy = model.evaluate(X_test, y_test)
print('MSE: %.2f' % (accuracy))

8254/8254 [==============================] - 1s 80us/step
MSE: 0.00


501